# 340B Site-of-Care Optimization Engine Demo

This notebook demonstrates the core functionality of the 340B Optimizer, which determines the optimal treatment pathway (Retail Pharmacy vs. Medical Billing) for 340B-eligible drugs.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/crowbarmassage/340b-optimizer/blob/main/notebooks/demo.ipynb)

## Step 1: Setup

Clone the repository and install dependencies.

In [ ]:
# Clone the repository
!git clone https://github.com/crowbarmassage/340b-optimizer.git
%cd 340b-optimizer

# Install dependencies
!pip install -e . -q
print("Setup complete!")

## Step 2: Import Libraries

Import the core modules from the 340B Optimizer.

In [ ]:
from decimal import Decimal
import polars as pl

from optimizer_340b import Drug, MarginAnalysis, DosingProfile
from optimizer_340b.compute.margins import (
    analyze_drug_margin,
    calculate_retail_margin,
    calculate_medicare_margin,
    calculate_commercial_margin,
)
from optimizer_340b.compute.dosing import calculate_year_1_vs_maintenance_delta

print("Imports successful!")

## Step 3: Create a Sample Drug

Let's create a sample drug (Humira-like profile) to demonstrate margin calculations.

In [ ]:
# Create a sample drug with realistic pricing
humira = Drug(
    ndc="0074-4339-02",
    drug_name="HUMIRA",
    manufacturer="ABBVIE",
    contract_cost=Decimal("150.00"),    # 340B acquisition cost
    awp=Decimal("6500.00"),              # Average Wholesale Price
    asp=Decimal("2800.00"),              # Average Sales Price
    hcpcs_code="J0135",                  # Medicare billing code
    bill_units_per_package=2,
    therapeutic_class="TNF Inhibitor",
    is_biologic=True,
)

print(f"Drug: {humira.drug_name}")
print(f"NDC: {humira.ndc}")
print(f"Contract Cost: ${humira.contract_cost}")
print(f"AWP: ${humira.awp}")
print(f"ASP: ${humira.asp}")
print(f"Has Medical Path: {humira.has_medical_path()}")

## Step 4: Calculate Margins

Calculate margins for all three pathways:
1. **Retail Pharmacy** - AWP-based with capture rate
2. **Medicare Medical** - ASP + 6%
3. **Commercial Medical** - ASP + 15%

In [ ]:
# Calculate individual margins
retail_gross, retail_net = calculate_retail_margin(humira)
medicare_margin = calculate_medicare_margin(humira)
commercial_margin = calculate_commercial_margin(humira)

print("=== MARGIN COMPARISON ===")
print()
print("RETAIL PATHWAY:")
print(f"  Formula: AWP × 85% × Capture Rate - Contract Cost")
print(f"  Gross Margin: ${retail_gross:,.2f}")
print(f"  Net Margin (45% capture): ${retail_net:,.2f}")
print()
print("MEDICARE MEDICAL PATHWAY:")
print(f"  Formula: ASP × 1.06 × Bill Units - Contract Cost")
print(f"  Margin: ${medicare_margin:,.2f}")
print()
print("COMMERCIAL MEDICAL PATHWAY:")
print(f"  Formula: ASP × 1.15 × Bill Units - Contract Cost")
print(f"  Margin: ${commercial_margin:,.2f}")

## Step 5: Get Full Analysis with Recommendation

Run the complete margin analysis to get the recommended pathway.

In [ ]:
# Perform full margin analysis
analysis = analyze_drug_margin(humira)

print("=== OPTIMIZATION RESULT ===")
print()
print(f"Drug: {analysis.drug.drug_name}")
print(f"NDC: {analysis.drug.ndc}")
print()
print(f"Retail Net Margin:     ${float(analysis.retail_net_margin):>10,.2f}")
print(f"Medicare Margin:       ${float(analysis.medicare_margin):>10,.2f}")
print(f"Commercial Margin:     ${float(analysis.commercial_margin):>10,.2f}")
print()
print(f"✅ RECOMMENDED PATH: {analysis.recommended_path.value}")
print(f"   Margin Delta: ${float(analysis.margin_delta):,.2f}")

## Step 6: Capture Rate Sensitivity Analysis

See how the recommendation changes as capture rate varies.

In [ ]:
print("=== CAPTURE RATE SENSITIVITY ===")
print()
print(f"{'Capture Rate':<15} {'Retail Margin':<15} {'Recommendation':<25}")
print("-" * 55)

for rate in [100, 80, 60, 45, 40, 20]:
    analysis = analyze_drug_margin(humira, capture_rate=Decimal(str(rate/100)))
    print(f"{rate}%{'':<13} ${float(analysis.retail_net_margin):>10,.2f}   {analysis.recommended_path.value}")

## Step 7: Loading Dose Analysis

Demonstrate Year 1 vs. Maintenance revenue for biologics with loading doses.

In [ ]:
# Create a dosing profile for Cosentyx (heavy loading dose)
cosentyx_dosing = DosingProfile(
    drug_name="COSENTYX",
    indication="Psoriasis",
    year_1_fills=17,           # Loading: 5 fills in month 1, then monthly
    year_2_plus_fills=12,      # Monthly maintenance
    adjusted_year_1_fills=Decimal("15.3"),  # 90% compliance
)

# Calculate revenue delta
margin_per_fill = Decimal("500.00")  # Example margin
revenue_analysis = calculate_year_1_vs_maintenance_delta(cosentyx_dosing, margin_per_fill)

print("=== LOADING DOSE IMPACT ===")
print()
print(f"Drug: {cosentyx_dosing.drug_name}")
print(f"Indication: {cosentyx_dosing.indication}")
print()
print(f"Year 1 Fills: {cosentyx_dosing.year_1_fills} (adjusted: {cosentyx_dosing.adjusted_year_1_fills})")
print(f"Maintenance Fills: {cosentyx_dosing.year_2_plus_fills}/year")
print()
print(f"Year 1 Revenue:      ${float(revenue_analysis['year_1_revenue']):>10,.2f}")
print(f"Maintenance Revenue: ${float(revenue_analysis['maintenance_revenue']):>10,.2f}")
print(f"Loading Dose Delta:  ${float(revenue_analysis['loading_dose_delta']):>10,.2f}")
print(f"Delta Percentage:    {float(revenue_analysis['loading_dose_delta_pct']):>10.1f}%")
print()
print("⚠️  Ignoring loading doses would underestimate Year 1 revenue significantly!")

## Step 8: Provenance Chain

Every calculation in the 340B Optimizer has a full provenance chain for auditability.

In [ ]:
# Get the display dictionary with all provenance fields
display_data = analysis.to_display_dict()

print("=== PROVENANCE CHAIN ===")
print()
for key, value in display_data.items():
    print(f"{key}: {value}")

## About

### Architecture

The 340B Optimizer uses a layered data architecture:

1. **Bronze Layer** - Raw data ingestion and validation
2. **Silver Layer** - Data normalization and crosswalk joins
3. **Gold Layer** - Margin calculations and recommendations
4. **Watchtower** - Risk flagging (IRA, Penny Pricing)

### Key Formulas

| Pathway | Formula |
|---------|--------|
| Retail | `AWP × 85% × Capture_Rate - Contract_Cost` |
| Medicare Medical | `ASP × 1.06 × Bill_Units - Contract_Cost` |
| Commercial Medical | `ASP × 1.15 × Bill_Units - Contract_Cost` |

### GitHub Repository

https://github.com/crowbarmassage/340b-optimizer